In [6]:
import numpy as np
from common.util import im2col, col2im
from common.functions import *

------

#### 계층의 `구현 규칙`
- 모든 계층은 `forward()`와 `backward()` 메서드를 가진다.
- 모든 계층은 인스턴스 변수인 `params`와 `grads`를 가진다.

`forward()` : 순전파 수행  
`backward()` : 역전파 수행  
`params` : 가중치와 편향 같은 매개변수를 담는 리스트  
`grads` : `params`에 저장된 각 매개변수에 대응하여, 해당 매개변수의 기울기를 저장하는 리스트

_여기서 연습으로 계층 코드를 작성하고 `common` 폴더에 `layers.py` 파일에 옮겨 적습니다._

---

### MatMul

<p align="center"><img src="./master/images/fig 1-25.png" width=300 />

In [10]:
class MatMul:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.x = None
        
    def forward(self, x):
        W, = self.params # tuple 인 self.params를 벗겨서  W에 받기 위해 W, 를 사용한다
        out = np.matmul(x, W)
        self.x = x
        return out
    
    def backward(self, dout):
        W, = self.params
        dx = np.matmul(dout, W.T)
        dW = np.matmul(self.x.T, dout)
        self.grads[0][...] = dW # self.grads[0] = dW 을 하게 되면 grads를 수정할 때 dW 값도 바뀌게 된다. 이를 막기 위해 self.grads[0][...]을 사용한다
        return dx

### Sigmoid

<p align="center"><img src="./master/images/fig 1-28.png" width=300 />

In [11]:
class Sigmoid:
    def __init__(self):
        self.params, self.grads = [], []
        self.out = None
        
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 -self.out) * self.out
        return dx


### Affine

<p align="center"><img src="./master/images/fig 1-29.png" width=300 />

In [12]:
class Affine: # TODO : MatMul class를 이용해서 구현하기
    def __init__(self, W, b):
        self.params = [W, b]
        self.grads = [np.zeros_like(W), np.zeros_like(b)]
        self.x = None
        
    def forward(self, x):
        W, b = self.params
        out = np.matmul(x, W) + b
        self.x = x
        return out
    
    def backward(self, dout):
        W, b = self.params
        dx = np.matmul(dout, W.T)
        dW = np.matmul(self.x.T, dout)
        db = np.sum(dout, axis=0)
        
        self.grads[0][...] = dW
        self.grads[1][...] = db
        
        return dx


### Softmax with Loss

<p align="center"><img src="softmaxwithloss_image.jpeg" />

In [7]:
class SoftmaxWithLoss:
    def __init__(self):
        self.params, self.grads = [], []
        self.y = None # Softmax의 출력
        self.t = None # 정답 레이블
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        
        # 정답 레이블이 원핫 벡터인 경우 정답의 인덱스로 변환
        if self.t.size == self.y.size:
            self.t = self.t.argmax(axis=1)
            
        loss = cross_entropy_error(self.y, self.t)
        return loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        
        dx = self.y.copy()
        dx[np.arange(batch_size), t] -= 1 # y - t 인데 t는 정답 인덱스에 1 값, 정답이 아닌 곳에는 0을 가지므로 1을 빼준다.
        dx *= dout
        dx = dx / batch_size 
        
        return dx